In [2]:
from langchain_community.document_loaders import JSONLoader
from pprint import pprint
loader = JSONLoader(
    file_path='./sample-data.json',
    jq_schema='.',
    text_content=False)

data = loader.load()
pprint(data)

[Document(metadata={'source': '/home/quan/Desktop/pgvector/sample-data.json', 'seq_num': 1}, page_content="[{'patient_name': 'Adam', 'patient_age': 16, 'patient_gender': 'male', 'medical_diagnosis': [{'name': 'fever'}, {'name': 'dizziness'}], 'medical_treatment': [{'name': 'medicines', 'prescription': ''}, {'name': 'orange juices', 'prescription': ''}], 'health_vital': [{'status': 'normal', 'value': '', 'units': ''}]}, {'patient_name': 'Bob', 'patient_age': 16, 'patient_gender': 'male', 'medical_diagnosis': [{'name': 'fever'}, {'name': 'dizziness'}], 'medical_treatment': [{'name': 'medicine', 'prescription': ''}], 'health_vital': [{'status': 'normal', 'value': '', 'units': ''}]}, {'patient_name': 'Craig', 'patient_age': 16, 'patient_gender': '', 'medical_diagnosis': [{'name': 'fever'}, {'name': 'headache'}], 'medical_treatment': [{'name': 'medicine', 'prescription': ''}, {'name': 'full meal per day', 'prescription': ''}], 'health_vital': [{'status': 'normal', 'value': '', 'units': ''}]

In [9]:

from psycopg2 import connect
 
# declare the connection string specifying
# the host name database name
# use name and password
conn_string = "host='localhost' \
dbname='patients' user='postgres'\
password='132456' port = '5432' "
 
# use connect function to establish the connection
conn = connect(conn_string)


In [33]:
import psycopg2

# Establishing the connection
conn = psycopg2.connect(
	database="patients",
	user='postgres',
	password='132456',
	host='localhost',
	port='5432'
)

# Creating a cursor object using the 
# cursor() method
cursor = conn.cursor()

# Doping EMPLOYEE table if already exists.
cursor.execute("DROP TABLE IF EXISTS patients")

# Creating table as per requirement
sql = '''CREATE TABLE patients(
                
                patient_name VARCHAR  PRIMARY KEY,
                patient_age INT NOT NULL,
                patient_gender VARCHAR,
                medical_diagnosis VARCHAR[],
                medical_treatment VARCHAR[],
                health_vital VARCHAR[]

)'''
cursor.execute(sql)
print("Table created successfully........")
conn.commit()

# Closing the connection
conn.close()


Table created successfully........


In [24]:
import random
id = random.randint(1, 200)
print(id) 

22


In [36]:
import json

conn = psycopg2.connect(
	database="patients",
	user='postgres',
	password='132456',
	host='localhost',
	port='5432'
)

cursor = conn.cursor()

postgres_insert_query = """ INSERT INTO patients(
    patient_name, patient_age, patient_gender, medical_diagnosis, medical_treatment, health_vital) 
    VALUES (%s,%s,%s,%s,%s,%s)"""

with open('sample-data.json') as data_file:    
    data = json.load(data_file)
    for item in data: 
        item['medical_diagnosis'] = json.dumps(item['medical_diagnosis'])
        item['medical_treatment'] = json.dumps(item['medical_treatment'])
        item['health_vital'] = json.dumps(item['health_vital'])
        cursor.execute(postgres_insert_query, (
                        item['patient_name'], 
                        item['patient_age'], 
                        item['patient_gender'], 
                        item['medical_diagnosis'], 
                        item['medical_treatment'], 
                        item['health_vital']
                    ))
        conn.commit()
        count = cursor.rowcount
    print(count, "Record inserted successfully \
    into patients table")

InvalidTextRepresentation: malformed array literal: "[{"name": "fever"}, {"name": "dizziness"}]"
LINE 3:     VALUES ('Adam',16,'male','[{"name": "fever"}, {"name": "...
                                     ^
DETAIL:  "[" must introduce explicitly-specified array dimensions.
